# Agents
- In LangChain, agents are systems that autonomously decide how to use tools to accomplish a task. They take an input, analyze it, and choose the right tool(s) to solve the problem.
- Types of Agents in LangChain:
# Zero-shot Agent:

- Does not rely on prior training or a pre-defined plan.
  Decides what to do based on the current input alone.
  
# Self-Ask Agent:

- Asks clarifying questions if needed before selecting a tool or performing an action.
  It’s like a problem solver that seeks more info to make better decisions.

# React Agent:

- Reacts based on the outcome of the previous tool or action.
  It adapts its next step depending on the result of its earlier choices.

# Agent with Memory:

- Remembers past interactions or decisions, which helps it make more informed choices in the future.



# 1. Zero-shot Agent
- Example:
    You ask, "What's the weather today in New York?"
    The zero-shot agent immediately chooses a weather API and calls it to get the answer without asking any questions or needing extra context.

# 2. Self-Ask Agent
-   Example:
    You ask, "Who won the last World Cup?"

    The agent first asks, "Which year are you referring to?" (since there have been multiple World Cups).
    Once the user provides the year, it fetches the correct result from the web or database.

# 3. React Agent
-   Example:
    You ask, "Translate 'Bonjour' to English."

    The agent uses a translation tool and gets the result "Hello."
    If the user then asks, "What language is that?", the agent can use another tool to identify the language and respond "French."

# 4. Agent with Memory
-   Example:
    You ask, "What’s my schedule for today?"

    The agent retrieves your calendar from a tool and lists your meetings.
    The next time you ask about your schedule, the agent remembers your previous meetings and can refer to them while answering new questions.
    In each case, the agent is smart enough to decide how to use tools and adapt based on the input or previous interactions.

In [1]:
# Zero Shot Agent

In [2]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents import create_openai_functions_agent
from langchain.llms import OpenAI

# Initialize the language model (OpenAI GPT)
llm = OpenAI(temperature=0)

# Create some sample tools (you can replace these with actual tools like APIs or databases)
tools = [
    Tool(
        name="Search",
        func=lambda x: f"Searching for {x}",  # Dummy function for illustration
        description="A tool to search for information"
    ),
    Tool(
        name="Calculator",
        func=lambda x: eval(x),  # Basic calculator tool
        description="A tool to perform mathematical operations"
    ),
]

# Initialize the zero-shot agent
agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example task: Using the zero-shot agent to handle a question
query  = "What is 2+2"
response = agent.run(query)

print(response)


C:\Users\Adari Shanmukh\AppData\Local\Temp\ipykernel_1020\2593219042.py:6: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
C:\Users\Adari Shanmukh\AppData\Local\Temp\ipykernel_1020\2593219042.py:23: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://pyth



> Entering new AgentExecutor chain...
 I should use a calculator to perform the mathematical operation.
Action: Calculator
Action Input: 2+2
Observation: 4
Thought: I now know the final answer.
Final Answer: 4

> Finished chain.
4


In [6]:
from langchain.agents import initialize_agent, Tool, AgentType

from langchain.agents import self_ask_with_search
from langchain.llms import OpenAI
from langchain.agents import create_openai_functions_agent
from langchain.tools import DuckDuckGoSearchResults

# Initialize the language model (e.g., OpenAI GPT)
llm = OpenAI(temperature=0)

# Set up a search tool (using DuckDuckGo search)
search_tool = DuckDuckGoSearchResults()

# Define a self-ask agent with search
tools = [
    Tool(
        name="Search",
        func=search_tool.run,  # DuckDuckGo search function
        description="Search the web for information."
    ),
]


# Initialize the Self-Ask Agent (with search)
agent = initialize_agent(tools, llm, verbose=True, agent_type="self-ask-with-search")

# Example task: ask the agent about a complex topic
query = "Who won last world cup"

# Run the agent
response = agent.run(query)

# Print the result
print(response)




> Entering new AgentExecutor chain...
 I should use the Search tool to find the answer
Action: Search
Action Input: 'Who won last world cup'
Observation: snippet: The FIFA World Cup, often called the World Cup, is an international association football competition among the senior men's national teams of the members of the Fédération Internationale de Football Association (), the sport's global governing body.The tournament has been held every four years since the inaugural tournament in 1930, with the exception of 1942 and 1946 due to the Second World ..., title: FIFA World Cup - Wikipedia, link: https://en.wikipedia.org/wiki/FIFA_World_Cup, snippet: FIFA(Federation Internationale de Football Association) is the worldwide governing organization of football. The FIFA World Cup started in 1930 and is held every four years. 32 teams qualified for the last FIFA World Cup. It is one of the biggest sporting events in the world watched by billions of people. Uruguay won the first edition of

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [8]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [3]:
tools = [get_word_length]
print(tools)


[StructuredTool(name='get_word_length', description='Returns the length of a word.', args_schema=<class 'langchain_core.utils.pydantic.get_word_length'>, func=<function get_word_length at 0x0000012F38A66A20>)]


In [ ]:
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

#https://medium.com/@shravankoninti/agent-tools-basic-code-using-langchain-50e13eb07d92
# Load environment variables from .env file
load_dotenv()


# Define a very simple tool function that returns the current time
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime  # Import datetime module to get current time

    now = datetime.datetime.now()  # Get current time
    return now.strftime("%I:%M %p")  # Format time in H:MM AM/PM format


# List of tools available to the agent
tools = [
    Tool(
        name="Time",  # Name of the tool
        func=get_current_time,  # Function that the tool will execute
        # Description of the tool
        description="Useful for when you need to know the current time",
    ),
]

# Pull the prompt template from the hub
# ReAct = Reason and Action
# https://smith.langchain.com/hub/hwchase17/react
prompt = hub.pull("hwchase17/react")

# Initialize a ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini", temperature=0
)

# Create the ReAct agent using the create_react_agent function
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

# Create an agent executor from the agent and tools
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

# Run the agent with a test query
response = agent_executor.invoke({"input": "What time is it?"})

# Print the response from the agent
print("response:", response)



> Entering new AgentExecutor chain...
I need to find out the current time.  
Action: Time  
Action Input: None  02:49 PMI now know the final answer.  
Final Answer: The current time is 02:49 PM.

> Finished chain.
response: {'input': 'What time is it?', 'output': 'The current time is 02:49 PM.'}


In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

# Define a simple tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny and warm."

tools = [
    Tool(name="Weather Info", func=get_weather, description="Provides the weather information of a city")
]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent_type="zero-shot-react-description")

# Test the agent
result = agent.run("What's the weather like in Paris?")
print(result)

C:\Users\Adari Shanmukh\AppData\Local\Temp\ipykernel_14900\2044509963.py:12: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
C:\Users\Adari Shanmukh\AppData\Local\Temp\ipykernel_14900\2044509963.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://p

The weather in Paris is sunny and warm.


In [15]:
from langchain.agents import create_conversational_agent, Tool
from langchain.llms import OpenAI

def check_order_status(order_id: str) -> str:
    return f"Order {order_id} is in transit."

tools = [
    Tool(name="Order Status Checker", func=check_order_status, description="Checks the status of an order.")
]

llm = OpenAI(temperature=0.7)
agent = create_conversational_agent(tools, llm)

# Simulating conversation
agent_memory = {}
agent.run("I would like to check the status of order #12345", memory=agent_memory)
agent.run("Can you remind me what my last query was?", memory=agent_memory)

ImportError: cannot import name 'create_conversational_agent' from 'langchain.agents' (d:\GenAI\venv\Lib\site-packages\langchain\agents\__init__.py)